In [1]:
import os
import pandas as pd
import datetime as dt
import numpy as np
import re

In [2]:
today = pd.Timestamp(dt.datetime.today().date())
today_ymd = today.strftime('%Y%m%d')
today_y_m_d = today.strftime('%Y_%m_%d')

yesterday = today - dt.timedelta(days=1)
yesterday_ymd = yesterday.strftime('%Y%m%d')
yesterday_y_m_d = yesterday.strftime('%Y-%m-%d')

tomorrow = today + dt.timedelta(days=1)
tomorrow_ymd = tomorrow.strftime('%Y%m%d')
tomorrow_y_m_d = tomorrow.strftime('%Y-%m-%d')

print(today_y_m_d)

2022_09_12


In [3]:
column_names = {'Emiasid': 'emiasid', 'Emiasid ': 'emiasid' , 'emiasid ': 'emiasid', 'EMIAS_ID': 'emiasid', 'МО': 'mo', 
                'МУ': 'mu', 'Логин оператора': 'login', 'Дата и время звонка': 'call_date', 'Результат звонка': 'call_result',
                'Результат звонка ': 'call_result', 'Разговор состоялся': 'talk_result', 'Комменатрий': 'comment', 
                'Дата и время переноса': 'next_call_date', 'Контактный телефон': 'phone_number', 'ФИО врача': 'doctor_fio', 
                'ФИО помощника врача': 'assistant_fio', 'Логин помощника врача': 'login', 'Зал №': 'hall_number',
                'Логин naumen': 'login', 'МО закрепления': 'mo', 'Отсутствует на рабочем месте': 'absence'}

In [4]:
# путь папки с данными по звонкам
calls_data_path = r'\Users\murad_satabaev\Documents\GitHub\final_framework'
calls_data_path = calls_data_path.split('\\')
calls_data_path = os.path.join('C:\\', r'\\', *calls_data_path)
calls_data_path = os.path.join(calls_data_path, 'calls_data_20220802_today.xlsx')

In [5]:
# путь файла с текущими статусами пациентов
patients_current_statuses_path = r'Users\murad_satabaev\Documents\GitHub\final_framework\current_statuses_path'
patients_current_statuses_path = patients_current_statuses_path.split('\\')
patients_current_statuses_path = os.path.join('C:\\', r'\\', *patients_current_statuses_path)
print(patients_current_statuses_path)

C:\\Users\murad_satabaev\Documents\GitHub\final_framework\current_statuses_path


In [6]:
calls_data_reading = pd.read_excel(calls_data_path)
calls_data_reading.shape[0]

252723

In [7]:
calls_data_reading = calls_data_reading.rename(columns=column_names)

In [9]:
calls_data_reading['emiasid'] = calls_data_reading['emiasid'].astype(str).str.replace('\.0', '')
calls_data_reading['call_date'] = calls_data_reading['call_date'].apply(lambda x: pd.to_datetime(x, dayfirst=True,
                                                                                                 errors='coerce').date())
calls_data_reading['next_call_date'] = \
calls_data_reading['next_call_date'].apply(lambda x: pd.to_datetime(x, dayfirst=True, errors='coerce').date())

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_1896\696863348.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  calls_data_reading['emiasid'] = calls_data_reading['emiasid'].astype(str).str.replace('\.0', '')


In [10]:
monthes = {'august', 'september'}

In [11]:
for month in monthes:
    
    
    if 'september' in month:
        calls_data = calls_data_reading[calls_data_reading.call_date >= dt.date(2022, 9, 2)].reset_index(drop=True).copy()
    else:
        calls_data = calls_data_reading.copy()
        
    
    emiasid_touches = \
    calls_data.groupby('emiasid').agg({'call_date': 'count'}).rename(columns={'call_date': 'calls_amount'}).reset_index()
    
    last_calls_data = \
    calls_data.sort_values(by='call_date').drop_duplicates(subset=['emiasid'], keep='last').reset_index(drop=True)
    
    last_calls_data = last_calls_data[['emiasid', 'call_date', 'next_call_date', 'comment', 'call_result', 'talk_result']]
    
    last_calls_data = last_calls_data.rename(
        columns={'comment': 'old_comment', 'call_date': 'last_call_date', 'call_result': 'last_call_result',
                'talk_result': 'last_talk_result'})
    
    last_calls_data['emiasid'] = last_calls_data['emiasid'].astype(str).str.replace('\.0', '')
    last_calls_data['last_call_date'] = last_calls_data['last_call_date'].apply(lambda x: pd.to_datetime(x, dayfirst=True).date())
    last_calls_data['next_call_date'] = \
    last_calls_data['next_call_date'].apply(lambda x: pd.to_datetime(x, dayfirst=True).date())
    
    
    patients_current_statuses_reading = pd.read_excel(
        os.path.join(patients_current_statuses_path, rf'{month}_patients_current_statuses_{today_y_m_d}.xlsx'))
    
    
    patients_current_statuses = patients_current_statuses_reading.rename(columns=column_names).copy()
    
    patients_current_statuses['emiasid'] = patients_current_statuses['emiasid'].astype(str).str.replace('\.0', '')
    patients_current_statuses['phone_number'] = patients_current_statuses['phone_number'] \
    .apply(lambda x: str(x).replace('.0', '') if not pd.isna(x) else x)
    
    
    patients_current_statuses_merged = patients_current_statuses.merge(emiasid_touches, on=['emiasid'], how='left')
    patients_current_statuses_merged = patients_current_statuses_merged.merge(last_calls_data, on=['emiasid'], how='left')
    
    if 'september' in month:
        september_patients_current_statuses_merged = patients_current_statuses_merged.copy()
    elif 'august' in month:
        august_patients_current_statuses_merged = patients_current_statuses_merged.copy()
    
    patients_current_statuses_merged.to_excel(
        os.path.join('C:\\', 'Users', 'murad_satabaev', 'Documents', 'GitHub',
        'final_framework', 'merged_statuses', 
                     f'{month}_patients_current_statuses_merged_with_calls_{today_y_m_d}.xlsx'), index=False)

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_1896\1341939309.py:22: FutureWarning: The default value of regex will change from True to False in a future version.
  last_calls_data['emiasid'] = last_calls_data['emiasid'].astype(str).str.replace('\.0', '')
C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_1896\1341939309.py:34: FutureWarning: The default value of regex will change from True to False in a future version.
  patients_current_statuses['emiasid'] = patients_current_statuses['emiasid'].astype(str).str.replace('\.0', '')
C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_1896\1341939309.py:22: FutureWarning: The default value of regex will change from True to False in a future version.
  last_calls_data['emiasid'] = last_calls_data['emiasid'].astype(str).str.replace('\.0', '')
C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_1896\1341939309.py:34: FutureWarning: The default value of regex will change from True to False in a future version.
  patients_current_st

In [12]:
# путь папки с раскреплением
assistant_to_doctor_distribution_path = \
r'\\t999\Сетевой диск\pdn\data\assistant_to_doctor_distribution_path'
assistant_to_doctor_distribution_path = \
assistant_to_doctor_distribution_path.split('\\')
assistant_to_doctor_distribution_path = \
os.path.join(r'\\', *assistant_to_doctor_distribution_path)
print(assistant_to_doctor_distribution_path)

\\t999\Сетевой диск\pdn\data\assistant_to_doctor_distribution_path


In [13]:
august_assistant_to_doctor_distribution_path = [os.path.join(assistant_to_doctor_distribution_path, filename) for filename 
                                         in os.listdir(assistant_to_doctor_distribution_path) if '~' not in filename
                                        and 'august' in filename]
august_assistant_to_doctor_distribution_path.sort(key=lambda x: os.path.getctime(x))
august_assistant_to_doctor_distribution_path = august_assistant_to_doctor_distribution_path[-1]
print(august_assistant_to_doctor_distribution_path)

\\t999\Сетевой диск\pdn\data\assistant_to_doctor_distribution_path\august_patient_to_assistant_distribution.xlsx


In [14]:
august_assistant_to_doctor_distribution = pd.read_excel(august_assistant_to_doctor_distribution_path, 
                                                       usecols=['login', 'mo', 'doctor_fio'])[['login', 'mo', 'doctor_fio']]

In [15]:
august_patients_current_statuses_merged_ = \
august_patients_current_statuses_merged.merge(august_assistant_to_doctor_distribution, on=['mo', 'doctor_fio'], how='left')

In [16]:
august_patients_current_statuses_merged_['month'] = 'august'
august_patients_current_statuses_merged_['problem'] = np.nan

In [17]:
august_patients_current_statuses_merged_ = august_patients_current_statuses_merged_[
['emiasid', 'login', 'status', 'old_comment', 'next_call_date', 
 'mo', 'doctor_fio',
 'phone_number', 'calls_amount', 'last_call_date', 'last_call_result', 'last_talk_result', 'problem', 'month']]

In [18]:
august_patients_current_statuses_merged_.to_excel(
        os.path.join('C:\\', 'Users', 'murad_satabaev', 'Documents', 'GitHub',
        'final_framework', 'merged_statuses',
                     f'august_patients_current_statuses_merged_with_logins_{today_y_m_d}.xlsx'), index=False)

In [19]:
september_assistant_to_doctor_distribution_path = [os.path.join(assistant_to_doctor_distribution_path, filename) for filename 
                                         in os.listdir(assistant_to_doctor_distribution_path) if '~' not in filename
                                        and 'september' in filename]
september_assistant_to_doctor_distribution_path.sort(key=lambda x: os.path.getctime(x))
september_assistant_to_doctor_distribution_path = september_assistant_to_doctor_distribution_path[-1]
print(september_assistant_to_doctor_distribution_path)

\\t999\Сетевой диск\pdn\data\assistant_to_doctor_distribution_path\september_patient_to_assistant_distribution.xlsx


In [20]:
september_assistant_to_doctor_distribution = pd.read_excel(september_assistant_to_doctor_distribution_path, 
                                    usecols=['login', 'emiasid', 'mo', 'doctor_fio'])[['emiasid', 'login', 'mo', 'doctor_fio']]

In [21]:
september_assistant_to_doctor_distribution['emiasid'] = \
september_assistant_to_doctor_distribution['emiasid'].astype(str).str.replace('\.0', '')
september_assistant_to_doctor_distribution = \
september_assistant_to_doctor_distribution.rename(columns={'mo': 'initial_mo', 'doctor_fio': 'initial_doctor_fio'})

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_1896\1912213584.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  september_assistant_to_doctor_distribution['emiasid'].astype(str).str.replace('\.0', '')


In [22]:
september_patients_current_statuses_merged_ = \
september_patients_current_statuses_merged.merge(september_assistant_to_doctor_distribution, on=['emiasid'], how='left')

In [23]:
# путь файла с сегодняшними 
emias_data_path = r'\\t999\Сетевой диск\pdn\data\emias_data\xlsx'
emias_data_path = emias_data_path.split('\\')
emias_data_path = os.path.join(r'\\', *emias_data_path)
print(emias_data_path)

\\t999\Сетевой диск\pdn\data\emias_data\xlsx


In [24]:
today_emias_data_path = [os.path.join(emias_data_path, filename) for filename in os.listdir(emias_data_path) if '4059' 
                         in filename and '~' not in filename and today.strftime('%Y-%m-%d') in filename][0]
print(today_emias_data_path)

\\t999\Сетевой диск\pdn\data\emias_data\xlsx\DAT_4059_выгрузка_списков_на_обзвон_помощниками_2022-09-12.xlsx


In [25]:
today_emias_data = pd.read_excel(today_emias_data_path, usecols=['emiasid'])
today_emias_data['emiasid'] = today_emias_data['emiasid'].astype(str).str.replace('\.0', '')

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_1896\2812670752.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  today_emias_data['emiasid'] = today_emias_data['emiasid'].astype(str).str.replace('\.0', '')


In [26]:
september_patients_current_statuses_merged_.loc[
    (september_patients_current_statuses_merged_.status == 'Отсутствует номер'),
    'problem'
] = 'Отсутствует номер'

In [27]:
september_patients_current_statuses_merged_.loc[
    (september_patients_current_statuses_merged_.doctor_fio.isna()),
    'problem'
] = 'Нет прикрепления к врачу'

In [28]:
september_patients_current_statuses_merged_.loc[
    (september_patients_current_statuses_merged_.mo != september_patients_current_statuses_merged_.initial_mo),
    'problem'
] = 'Смена МО на другую нашу'

In [29]:
september_patients_current_statuses_merged_.loc[
    (september_patients_current_statuses_merged_.mo.isna()),
    'problem'
] = 'Нет прикрепления к МО'

In [30]:
september_patients_current_statuses_merged_.loc[
    (september_patients_current_statuses_merged_.status == 'Смена МО'),
    'problem'
] = 'Смена МО'

In [31]:
september_patients_current_statuses_merged_.loc[
    (~(september_patients_current_statuses_merged_.emiasid.isin(today_emias_data.emiasid))),
    'problem'
] = 'Нет в сегодняшней выгрузке'

In [32]:
september_patients_current_statuses_merged_['month'] = 'september'

In [33]:
september_patients_current_statuses_merged__ = september_patients_current_statuses_merged_[
['emiasid', 'login', 'status', 'old_comment', 'next_call_date', 
 'mo', 'doctor_fio', 
#  'initial_mo', 'initial_doctor_fio',
 'phone_number', 'calls_amount', 'last_call_date', 'last_call_result', 'last_talk_result', 
 'problem', 'month']]

In [34]:
september_patients_current_statuses_merged__.to_excel(
        os.path.join('C:\\', 'Users', 'murad_satabaev', 'Documents', 'GitHub',
        'final_framework', 'merged_statuses', 
                     f'september_patients_current_statuses_merged_with_logins_{today_y_m_d}.xlsx'), index=False)

In [35]:
september_patients_current_statuses_merged_.loc[
    (september_patients_current_statuses_merged_.mo != september_patients_current_statuses_merged_.initial_mo),
    'login'
] = np.nan

In [36]:
september_patients_current_statuses_merged_.loc[
    (september_patients_current_statuses_merged_.status.isin({'Смена МО на другую нашу', 'Нет прикрепления к МО',
                                                             'Смена МО'})),
    'login'
] = np.nan

In [37]:
total_patients_current_statuses_merged__ = \
pd.concat([august_patients_current_statuses_merged_, september_patients_current_statuses_merged__]).reset_index(drop=True)

In [39]:
total_patients_current_statuses_merged__.to_excel(
        os.path.join('C:\\', 'Users', 'murad_satabaev', 'Documents', 'GitHub',
        'final_framework', 'merged_statuses', 
                     f'total_patients_current_statuses_merged_with_logins_{today_y_m_d}.xlsx'), index=False)